In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib as plt
import seaborn as sns
import cv2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from skin_cancer_detection.hairremoval import hair_removal

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install pickle5

In [ ]:
import pickle5 as pickle
with open('/content/drive/My Drive/skin_pickle.pickle', "rb") as fh:
    skin_df = pickle.load(fh)

In [ ]:
skin_df

In [ ]:
#check if images are displayed 
import matplotlib.pyplot as plt
plt.imshow(np.array(skin_df['image_resized'][0]))

In [ ]:
y = skin_df['dx']

In [ ]:
dict_target = {'bkl':0, 'nv':1, 'df':2, 'mel':3, 'vasc':4, 'bcc':5, 'akiec':6}

In [ ]:
y_num = y.map(dict_target.get)

In [ ]:
skin_df['target'] = y_num

In [ ]:
skin_df

In [ ]:
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y_num, num_classes = 7)

In [ ]:
y_cat.shape

In [ ]:
X = skin_df['image_resized']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3, random_state=42)

In [ ]:
X_train_stack = np.stack(X_train)

In [ ]:
X_test_stack = np.stack(X_test)

In [ ]:
X_train_stack.shape

## Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Reshape

#Basic CNN 1

model = Sequential()
model.add(layers.experimental.preprocessing.Rescaling(1. / 255))
model.add(layers.Conv2D(16, (3,3), input_shape=(75, 100, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(32, (2,2), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu')) # intermediate layer
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
#Example for model.compile and fit

from tensorflow.keras.metrics import Recall, Precision

model.compile(
  optimizer = 'adam',
  loss = 'categorical_crossentropy', # "sparse_" allows to avoid one-hot-encoding the target
  metrics = ['accuracy','Recall', 'Precision'])

model.fit(
    X_train_stack,y_train,
    validation_split=0.2,
    epochs=30,
    batch_size = 32
)

In [ ]:
# RENAME JOBLIB MODEL FILE!!!!

import joblib
joblib.dump(model, 'model.joblib')